# SMC Debug Notebook

Side-by-side comparison of baseline IB strategy vs SMC-enhanced strategy.

Usage:
1. Set symbol and date in Config cell
2. Run All Cells
3. Compare baseline vs SMC charts
4. Inspect SMC event log and registry

In [ ]:
# Cell 1: Setup
import sys
import logging
from pathlib import Path
from datetime import datetime

DUAL_V4_PATH = Path(".").parent.resolve()
sys.path.insert(0, str(DUAL_V4_PATH))

# Suppress noisy loggers
logging.basicConfig(level=logging.WARNING)
logging.getLogger("src.strategies").setLevel(logging.INFO)

from notebooks.debug_tools import SingleDayBacktest, SingleDayBacktestSMC

print(f"DUAL_V4_PATH: {DUAL_V4_PATH}")
print("Setup complete")

In [ ]:
# Cell 2: Configuration
SYMBOL = "GER40"
DATE = "2023-01-05"   # Known Reverse signal day
RISK_AMOUNT = 500.0

# SMC config overrides (empty dict = defaults)
SMC_CONFIG = {
    "enable_fractals": True,
    "enable_fvg": True,
    "enable_cisd": False,   # Phase 2 detectors
    "enable_bos": False,    # Phase 2 detectors
}

print(f"Symbol: {SYMBOL}")
print(f"Date: {DATE}")
print(f"SMC config: {SMC_CONFIG}")

In [ ]:
# Cell 3: Initialize baseline backtest
baseline = SingleDayBacktest(
    symbol=SYMBOL,
    risk_amount=RISK_AMOUNT,
)
print("Baseline backtest initialized")

In [ ]:
# Cell 4: Initialize SMC backtest
smc_bt = SingleDayBacktestSMC(
    symbol=SYMBOL,
    risk_amount=RISK_AMOUNT,
    smc_config=SMC_CONFIG,
)
print("SMC backtest initialized")

In [ ]:
# Cell 5: Run baseline
result_base = baseline.run_day(DATE)

if result_base["error"]:
    print(f"Baseline error: {result_base['error']}")
elif result_base["trade"]:
    t = result_base["trade"]
    print(f"Baseline: {t.variation} {t.direction.upper()} | "
          f"Entry: {t.entry_price:.2f} | SL: {t.sl:.2f} | TP: {t.tp:.2f} | "
          f"P/L: ${t.profit:.2f} | Exit: {t.exit_reason}")
else:
    print("Baseline: No trade")

In [ ]:
# Cell 6: Run SMC
result_smc = smc_bt.run_day(DATE)

if result_smc["error"]:
    print(f"SMC error: {result_smc['error']}")
elif result_smc["trade"]:
    t = result_smc["trade"]
    print(f"SMC: {t.variation} {t.direction.upper()} | "
          f"Entry: {t.entry_price:.2f} | SL: {t.sl:.2f} | TP: {t.tp:.2f} | "
          f"P/L: ${t.profit:.2f} | Exit: {t.exit_reason}")
else:
    print("SMC: No trade")

# SMC decision
dec = result_smc.get("smc_decision")
if dec:
    print(f"\nSMC Decision: {dec.action} | Score: {dec.confluence_score:.1f} | {dec.reason}")
else:
    print("\nNo SMC decision (signal may not have been evaluated)")

In [ ]:
# Cell 7: Comparison summary
base_trade = result_base.get("trade")
smc_trade = result_smc.get("trade")

print("=" * 60)
print(f"{'':20s} {'BASELINE':>15s}   {'SMC':>15s}")
print("=" * 60)

def _fmt(val, fmt=".2f"):
    return f"{val:{fmt}}" if val is not None else "-"

rows = [
    ("Variation", getattr(base_trade, "variation", "-"), getattr(smc_trade, "variation", "-")),
    ("Direction", getattr(base_trade, "direction", "-"), getattr(smc_trade, "direction", "-")),
    ("Entry Price", _fmt(getattr(base_trade, "entry_price", None)), _fmt(getattr(smc_trade, "entry_price", None))),
    ("Stop Loss", _fmt(getattr(base_trade, "sl", None)), _fmt(getattr(smc_trade, "sl", None))),
    ("Take Profit", _fmt(getattr(base_trade, "tp", None)), _fmt(getattr(smc_trade, "tp", None))),
    ("Exit Reason", getattr(base_trade, "exit_reason", "-"), getattr(smc_trade, "exit_reason", "-")),
    ("P/L ($)", _fmt(getattr(base_trade, "profit", None)), _fmt(getattr(smc_trade, "profit", None))),
]

for label, v1, v2 in rows:
    match = " " if str(v1) == str(v2) else "*"
    print(f"{label:20s} {str(v1):>15s}   {str(v2):>15s}  {match}")

print("=" * 60)
print("* = difference between baseline and SMC")

In [ ]:
# Cell 8: Baseline chart
%matplotlib inline
fig_base = baseline.generate_chart(result_base)
if fig_base:
    fig_base.set_size_inches(14, 8)

In [ ]:
# Cell 9: SMC chart (with overlays)
%matplotlib inline
fig_smc = smc_bt.generate_chart_with_smc(result_smc)
if fig_smc:
    fig_smc.set_size_inches(16, 10)

In [ ]:
# Cell 10: SMC Event Log
events = result_smc.get("smc_event_log", [])
print(f"SMC Events: {len(events)}")
print("-" * 80)

for evt in events:
    ts = evt.timestamp.strftime("%H:%M:%S") if evt.timestamp else "?"
    price_str = f"@ {evt.price:.2f}" if evt.price else ""
    dir_str = f" {evt.direction}" if evt.direction else ""
    print(f"  {ts}  {evt.event_type:25s} {evt.timeframe:4s}{dir_str:10s} {price_str}")

In [ ]:
# Cell 11: Registry snapshot
smc_state = result_smc.get("smc_state")
if smc_state:
    print(f"Registry count: {smc_state.get('registry_count', 0)}")
    print(f"Event count: {smc_state.get('event_count', 0)}")
    print(f"Market structures: {smc_state.get('market_structures', {})}")
else:
    print("No SMC state available (engine may not have initialized)")

# Detailed registry dump
engine = getattr(smc_bt.strategy, "smc_engine", None)
if engine:
    for stype in ["fractal", "fvg", "bos", "cisd"]:
        df = engine.registry.to_dataframe(stype)
        if not df.empty:
            print(f"\n--- {stype.upper()} ({len(df)}) ---")
            display(df)
        else:
            print(f"\n--- {stype.upper()}: none ---")

In [ ]:
# Cell 12: Multi-day comparison (optional)
# Uncomment and edit dates to run a batch comparison

# DATES = ["2023-01-05", "2023-01-06", "2023-02-14"]
# 
# for d in DATES:
#     r_base = baseline.run_day(d)
#     r_smc = smc_bt.run_day(d)
#     
#     t_base = r_base.get("trade")
#     t_smc = r_smc.get("trade")
#     dec = r_smc.get("smc_decision")
#     
#     base_pnl = f"${t_base.profit:.2f}" if t_base else "no trade"
#     smc_pnl = f"${t_smc.profit:.2f}" if t_smc else "no trade"
#     smc_action = dec.action if dec else "-"
#     
#     print(f"{d}  Base: {base_pnl:>12s}  SMC: {smc_pnl:>12s}  Decision: {smc_action}")